In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn


In [2]:
df = pd.read_csv('wptw.csv')
df

C:\Users\zisha\AppData\Local\Temp\ipykernel_21064\1743840569.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wptw.csv')


STORE         UPC  WEEK  MOVE  PRICE  QTY SALE  PROFIT  OK  \
0            2  1122542322   336     0   0.00    1  NaN    0.00   1   
1            2  1122542322   337     4   1.59    1  NaN    6.60   1   
2            2  1122542322   338    22   1.59    1  NaN    6.60   1   
3            2  1122542322   339    36   1.59    1  NaN    6.60   1   
4            2  1122542322   344    29   1.79    1  NaN   22.78   1   
...        ...         ...   ...   ...    ...  ...  ...     ...  ..   
1847012    139  7572536605   395     0   0.00    1  NaN    0.00   1   
1847013    139  7572536605   396     0   0.00    1  NaN    0.00   1   
1847014    139  7572536605   397     0   0.00    1  NaN    0.00   1   
1847015    139  7572536605   398     0   0.00    1  NaN    0.00   1   
1847016    139  7572536605   399     0   0.00    1  NaN    0.00   1   

                PRICE_HEX        PROFIT_HEX  
0        0000000000000000  0000000000000000  
1        3FF970A3D70A3D71  401A666666666666  
2        3FF970A3D70A3D71  401A666666666666  
3        3FF970A3D70A3D71  401A666666666666  
4        3FFCA3D70A3D70A4  4036C7AE147AE148  
...                   ...               ...  
1847012  0000000000000000  0000000000000000  
1847013  0000000000000000  0000000000000000  
1847014  0000000000000000  0000000000000000  
1847015  0000000000000000  0000000000000000  
1847016  0000000000000000  0000000000000000  

[1847017 rows x 11 columns]

In [3]:
selected_df = df[(df['UPC'] == 3700063527) & (df['STORE'] == 112)][['WEEK', 'MOVE']]

In [4]:
# Remove any noise
selected_df['MOVE'] = selected_df['MOVE'][(selected_df['MOVE'] - selected_df['MOVE'].mean()).abs() < 3 * selected_df['MOVE'].std()]
selected_df = selected_df.dropna()

In [5]:
# Add missing weeks
max_week = selected_df['WEEK'].max()
all_weeks = range(1, max_week + 1)
missing_weeks = set(all_weeks) - set(selected_df['WEEK'])
missing_data = [{'WEEK': week, 'MOVE': selected_df['MOVE'].mean()} for week in missing_weeks]
missing_df = pd.DataFrame(missing_data)
selected_df = pd.concat([selected_df, missing_df]).sort_values('WEEK').reset_index(drop=True)
selected_df['MOVE'] = selected_df['MOVE'].fillna(selected_df['MOVE'].mean())

In [6]:
data = selected_df

In [7]:
print(data.head(30))

    WEEK   MOVE
0      1   85.0
1      2   89.0
2      3   98.0
3      4   75.0
4      5  114.0
5      6   10.0
6      7  123.0
7      8  103.0
8      9   98.0
9     10  113.0
10    11   56.0
11    12   91.0
12    13   85.0
13    14  106.0
14    15  111.0
15    16   16.0
16    17   91.0
17    18   89.0
18    19   80.0
19    20   66.0
20    21   91.0
21    22   77.0
22    23   95.0
23    24   93.0
24    25   76.0
25    26   53.0
26    27   67.0
27    28   70.0
28    29   19.0
29    30   74.0


In [8]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [9]:
plt.figure(figsize=(12, 6))
plt.plot(data['WEEK'], data['MOVE'], color = '#6699ff')

In [10]:
from copy import deepcopy as dc
import pandas as pd

def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)
    
    df.set_index('WEEK', inplace=True) 
    
    for i in range(1, n_steps+1):
        df[f'Move(t-{i})'] = df['MOVE'].shift(i)
        
    df.dropna(inplace=True)
    
    return df

lookback = 7
shifted_df = prepare_dataframe_for_lstm(data, lookback)
shifted_df


MOVE  Move(t-1)  Move(t-2)   Move(t-3)   Move(t-4)   Move(t-5)  \
WEEK                                                                    
8     103.0      123.0       10.0  114.000000   75.000000   98.000000   
9      98.0      103.0      123.0   10.000000  114.000000   75.000000   
10    113.0       98.0      103.0  123.000000   10.000000  114.000000   
11     56.0      113.0       98.0  103.000000  123.000000   10.000000   
12     91.0       56.0      113.0   98.000000  103.000000  123.000000   
...     ...        ...        ...         ...         ...         ...   
395    70.0       45.0       46.0   60.152455   51.000000   43.000000   
396    50.0       70.0       45.0   46.000000   60.152455   51.000000   
397    44.0       50.0       70.0   45.000000   46.000000   60.152455   
398    59.0       44.0       50.0   70.000000   45.000000   46.000000   
399    56.0       59.0       44.0   50.000000   70.000000   45.000000   

       Move(t-6)   Move(t-7)  
WEEK                          
8      89.000000   85.000000  
9      98.000000   89.000000  
10     75.000000   98.000000  
11    114.000000   75.000000  
12     10.000000  114.000000  
...          ...         ...  
395    48.000000   42.000000  
396    43.000000   48.000000  
397    51.000000   43.000000  
398    60.152455   51.000000  
399    46.000000   60.152455  

[392 rows x 8 columns]

In [11]:
shifted_df_as_np = shifted_df.to_numpy()

shifted_df_as_np

array([[103.        , 123.        ,  10.        , ...,  98.        ,
         89.        ,  85.        ],
       [ 98.        , 103.        , 123.        , ...,  75.        ,
         98.        ,  89.        ],
       [113.        ,  98.        , 103.        , ..., 114.        ,
         75.        ,  98.        ],
       ...,
       [ 44.        ,  50.        ,  70.        , ...,  60.15245478,
         51.        ,  43.        ],
       [ 59.        ,  44.        ,  50.        , ...,  46.        ,
         60.15245478,  51.        ],
       [ 56.        ,  59.        ,  44.        , ...,  45.        ,
         46.        ,  60.15245478]])

In [12]:
shifted_df_as_np.shape

(392, 8)

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
shifted_df_as_np = scaler.fit_transform(shifted_df_as_np)

shifted_df_as_np

array([[ 0.57251908,  0.8778626 , -0.84732824, ...,  0.49618321,
         0.35877863,  0.29770992],
       [ 0.49618321,  0.57251908,  0.8778626 , ...,  0.14503817,
         0.49618321,  0.35877863],
       [ 0.72519084,  0.49618321,  0.57251908, ...,  0.74045802,
         0.14503817,  0.49618321],
       ...,
       [-0.32824427, -0.23664122,  0.06870229, ..., -0.08164191,
        -0.22137405, -0.34351145],
       [-0.09923664, -0.32824427, -0.23664122, ..., -0.29770992,
        -0.08164191, -0.22137405],
       [-0.14503817, -0.09923664, -0.32824427, ..., -0.3129771 ,
        -0.29770992, -0.08164191]])

In [14]:
X = shifted_df_as_np[:, 1:]
y = shifted_df_as_np[:, 0]

X.shape, y.shape

((392, 7), (392,))

In [15]:
X = dc(np.flip(X, axis=1))
X

array([[ 0.29770992,  0.35877863,  0.49618321, ...,  0.74045802,
        -0.84732824,  0.8778626 ],
       [ 0.35877863,  0.49618321,  0.14503817, ..., -0.84732824,
         0.8778626 ,  0.57251908],
       [ 0.49618321,  0.14503817,  0.74045802, ...,  0.8778626 ,
         0.57251908,  0.49618321],
       ...,
       [-0.34351145, -0.22137405, -0.08164191, ..., -0.3129771 ,
         0.06870229, -0.23664122],
       [-0.22137405, -0.08164191, -0.29770992, ...,  0.06870229,
        -0.23664122, -0.32824427],
       [-0.08164191, -0.29770992, -0.3129771 , ..., -0.23664122,
        -0.32824427, -0.09923664]])

In [16]:
split_index = int(len(X) * 0.80)

split_index

313

In [17]:
X_train = X[:split_index]
X_test = X[split_index:]

y_train = y[:split_index]
y_test = y[split_index:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((313, 7), (79, 7), (313,), (79,))

In [18]:
X_train = X_train.reshape((-1, lookback, 1))
X_test = X_test.reshape((-1, lookback, 1))

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((313, 7, 1), (79, 7, 1), (313, 1), (79, 1))

In [19]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([313, 7, 1]),
 torch.Size([79, 7, 1]),
 torch.Size([313, 1]),
 torch.Size([79, 1]))

In [20]:
from torch.utils.data import Dataset

class SalesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
train_dataset = SalesDataset(X_train, y_train)
test_dataset = SalesDataset(X_test, y_test)

In [21]:
train_dataset

In [22]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([16, 7, 1]) torch.Size([16, 1])


In [24]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_stacked_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

model = SimpleRNN(1, 4, 1)
model.to(device)
model

SimpleRNN(
  (rnn): RNN(1, 4, batch_first=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [25]:
import matplotlib.pyplot as plt

train_losses = []
val_losses = []

def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 10 == 9:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index + 1, avg_loss_across_batches))
            running_loss = 0.0
    train_losses.append(avg_loss_across_batches)
    print()

def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    val_losses.append(avg_loss_across_batches)
    print('***************************************************')
    print()

learning_rate = 0.001
num_epochs = 30
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

# Plotting
epochs = range(1, num_epochs + 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Epoch: 1
Batch 10, Loss: 0.031
Batch 20, Loss: 0.026

Val Loss: 0.169
***************************************************

Epoch: 2
Batch 10, Loss: 0.023
Batch 20, Loss: 0.020

Val Loss: 0.108
***************************************************

Epoch: 3
Batch 10, Loss: 0.017
Batch 20, Loss: 0.015

Val Loss: 0.065
***************************************************

Epoch: 4
Batch 10, Loss: 0.014
Batch 20, Loss: 0.011

Val Loss: 0.040
***************************************************

Epoch: 5
Batch 10, Loss: 0.011
Batch 20, Loss: 0.009

Val Loss: 0.028
***************************************************

Epoch: 6
Batch 10, Loss: 0.010
Batch 20, Loss: 0.008

Val Loss: 0.024
***************************************************

Epoch: 7
Batch 10, Loss: 0.009
Batch 20, Loss: 0.007

Val Loss: 0.022
***************************************************

Epoch: 8
Batch 10, Loss: 0.007
Batch 20, Loss: 0.009

Val Loss: 0.022
***************************************************

Epoch: 9
Batch 1

In [26]:
with torch.no_grad():
    predicted = model(X_train.to(device)).to('cpu').numpy()

plt.figure(figsize=(12, 8))
plt.plot(y_train, label='Moves', color = '#6699ff')
plt.plot(predicted, label='Forecasted', color = 'green')
plt.xlabel('Week')
plt.ylabel('Moves Over Time')
plt.legend()
plt.show()

In [27]:
train_predictions = predicted.flatten()

dummies = np.zeros((X_train.shape[0], lookback+1))
dummies[:, 0] = train_predictions
dummies = scaler.inverse_transform(dummies)

train_predictions = dc(dummies[:, 0])
train_predictions

array([ 87.20706549,  91.69963926,  93.19506344,  98.67945927,
        80.20955989,  85.49428031,  85.47317667,  93.15087874,
        97.75178555,  61.07390971,  74.77227195,  80.22915849,
        79.37270343,  72.62562855,  80.02800928,  77.23651814,
        85.81884606,  88.02093458,  82.36092238,  68.83296892,
        67.60418969,  67.05386667,  46.77346823,  58.03221332,
        61.93289072,  70.29779566,  76.11154075,  84.2449611 ,
        80.03808285,  79.52953325,  83.15671742,  95.04041834,
        94.68616773,  85.78365056,  91.63349593, 102.75724539,
        93.93247065,  91.87613796,  90.72383964,  95.76813751,
        96.45926352,  86.4484141 ,  83.77870564,  91.93976705,
        88.65401241,  91.36362863,  91.49747889,  88.25531553,
        86.20216468,  78.71706964,  54.62595119,  58.75238586,
        61.024718  ,  64.54621318,  69.94446788,  79.37846784,
        76.4424292 ,  83.8901073 ,  81.46209413,  91.70986801,
        80.00744246,  74.7179766 ,  72.60629445,  64.47

In [28]:
dummies = np.zeros((X_train.shape[0], lookback+1))
dummies[:, 0] = y_train.flatten()
dummies = scaler.inverse_transform(dummies)

new_y_train = dc(dummies[:, 0])
new_y_train

array([102.99999869,  98.00000069, 112.9999986 ,  55.99999969,
        91.00000036,  84.99999924, 105.99999827, 111.00000018,
        15.99999908,  91.00000036,  88.99999999,  80.00000026,
        66.        ,  91.00000036,  76.9999997 ,  94.99999915,
        93.00000073,  75.99999952,  53.00000011,  67.00000004,
        69.99999986,  18.99999866,  74.00000012,  72.99999993,
        84.99999924,  86.0000004 ,  96.00000031,  75.99999952,
        76.9999997 ,  88.00000077, 113.99999976,  99.00000185,
        74.00000012,  99.99999911, 131.        ,  86.0000004 ,
        88.99999999,  89.9999992 , 105.99999827, 102.00000143,
        72.99999993,  80.00000026, 103.99999985,  86.0000004 ,
        96.00000031,  93.99999994,  84.99999924,  83.00000082,
        68.99999992,  25.99999899,  65.        ,  68.99999992,
        74.00000012,  81.00000045,  93.99999994,  75.00000031,
        91.99999957,  79.00000007, 105.99999827,  66.        ,
        67.99999998,  72.00000023,  56.99999988,  91.00

In [29]:
plt.figure(figsize=(12, 8))
plt.plot(new_y_train, label='Moves', color = '#6699ff')
plt.plot(train_predictions, label='Forecasted', color ='green')
plt.xlabel('Week')
plt.ylabel('Moves Over Time')
plt.legend()
plt.show()

In [30]:
test_predictions = model(X_test.to(device)).detach().cpu().numpy().flatten()

dummies = np.zeros((X_test.shape[0], lookback+1))
dummies[:, 0] = test_predictions
dummies = scaler.inverse_transform(dummies)

test_predictions = dc(dummies[:, 0])
test_predictions

array([50.25511324, 50.61229974, 50.48495372, 61.47483486, 53.2252679 ,
       54.47977567, 49.02201168, 52.8504329 , 55.66089799, 52.59681644,
       52.33031642, 51.55807289, 55.95254433, 54.94096945, 55.76808907,
       53.39098541, 50.114382  , 51.08913141, 54.16202258, 55.07613344,
       55.75647631, 56.11690517, 56.39948618, 56.66318695, 58.3446578 ,
       62.11158743, 60.71478092, 59.36893957, 59.90852484, 68.65937439,
       66.68193584, 63.73186877, 62.36894989, 61.20850858, 63.2126678 ,
       68.2935486 , 62.67118709, 60.23043775, 68.56426016, 66.6191715 ,
       61.95035573, 58.06533672, 55.04975341, 61.42867371, 59.02965339,
       71.63593129, 66.24832455, 64.59635555, 61.31991025, 54.35622638,
       55.84393801, 56.27545279, 56.87924984, 57.02965936, 56.74939738,
       53.59732513, 60.66170365, 64.53386645, 63.89342646, 58.13351409,
       49.28357105, 47.84105334, 51.09439805, 54.26095843, 55.3935664 ,
       51.32417801, 48.62172583, 49.46655446, 47.18857601, 48.18

In [31]:
dummies = np.zeros((X_test.shape[0], lookback+1))
dummies[:, 0] = y_test.flatten()
dummies = scaler.inverse_transform(dummies)

new_y_test = dc(dummies[:, 0])
new_y_test

array([50.99999974, 50.99999974, 80.00000026, 46.00000076, 58.00000007,
       42.00000001, 58.99999977, 63.00000002, 50.99999974, 54.0000003 ,
       51.99999993, 65.        , 56.99999988, 59.99999995, 53.00000011,
       46.00000076, 53.00000011, 61.00000014, 59.99999995, 60.15245473,
       60.15245473, 60.15245473, 60.15245473, 63.99999996, 71.00000005,
       63.00000002, 61.00000014, 63.99999996, 83.00000082, 67.99999998,
       63.00000002, 63.99999996, 63.00000002, 68.99999992, 77.99999989,
       60.15245473, 60.15245473, 81.99999966, 67.99999998, 58.99999977,
       55.99999969, 54.0000003 , 72.99999993, 60.15245473, 91.00000036,
       63.99999996, 65.        , 59.99999995, 46.99999997, 60.15245473,
       60.15245473, 62.00000008, 61.00000014, 59.99999995, 51.99999993,
       72.99999993, 74.00000012, 67.00000004, 54.0000003 , 37.00000006,
       42.99999923, 55.99999969, 62.00000008, 61.00000014, 47.99999918,
       44.00000039, 49.99999955, 42.00000001, 47.99999918, 42.99

In [32]:
plt.figure(figsize=(12, 6))
plt.plot(new_y_test, label='Moves',color = '#6699ff')
plt.plot(test_predictions, label='Forecasted', color = 'green')
plt.xlabel('Week')
plt.ylabel('Moves Over Time')
plt.title("Simple RNN Model")
plt.legend()
plt.show()

In [33]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
# report performance
mse = mean_squared_error(new_y_test, test_predictions)
print('MSE: '+str(mse))
mae = mean_absolute_error(new_y_test, test_predictions)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(new_y_test, test_predictions))
print('RMSE: '+str(rmse))

MSE: 100.72144153533988
MAE: 7.509953872925496
RMSE: 10.036007250661982
